In [ ]:
# !pip install opencv-python
# install and configure ffmpeg on the system

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os
import requests
import IPython.display as ipd
import subprocess

In [ ]:
VIDEO_TYPE = {
    '.avi': cv2.VideoWriter_fourcc(*'XVID'),
    '.mp4': cv2.VideoWriter_fourcc(*'H264'),
}

In [ ]:
input_file = input("Enter the name of Raw Video: ")
subprocess.run(['ffmpeg', '-i', input_file,'-qscale', '0', 'process.mp4', '-loglevel', 'quiet'])

In [ ]:
audio_file = 'audio.mp3' 
subprocess.run(['ffmpeg', '-i', input_file, '-q:a', '0', audio_file, '-loglevel', 'quiet'])

In [ ]:
ipd.Video('process.mp4')

In [ ]:
def image_resize(image, width = None, height = None, inter = cv2.INTER_AREA):
    dim = None
    (h, w) = image.shape[:2]
    
    if width is None and height is None:
        return image
    if width is None:
        r = height / float(h)
        dim = (int(w * r), height)
    else:
        r = width / float(w)
        dim = (width, int(h * r))

    resized = cv2.resize(image, dim, interpolation = inter)
    return resized

def hollow_watermark(Text):
    fig, ax = plt.subplots(figsize=(6, 2))
    ax.text(0.5, 0.5, Text, fontsize=36, ha="center", va="center")
    ax.axis("off")
    plt.savefig("watermark.png", bbox_inches="tight", pad_inches=0)
    
def filled_watermark(Text):
    fig, ax = plt.subplots(figsize=(6, 2))
    ax.text(0.5, 0.5, Text, fontsize=36, ha="center", va="center", color='white')
    ax.axis("off")
    plt.savefig("watermark.png", bbox_inches="tight", pad_inches=0, facecolor="black")
    
def txtwatermark_type():
    while True:
        print("Choose a watermark text type:")
        print("1. Hollow Watermark")
        print("2. Filled Watermark")
        choice = int(input("Enter the option (1 or 2): "))
        if choice == 1 or choice == 2:
            break
        else:
            print("Invalid Input. Try again.")
    return choice

def txtwatermark_placement():
    while True:
        print("Choose where you want to place the watermark:")
        print("1. Bottom-Right Corner")
        print("2. Horizontal Watermark")
        choice = int(input("Enter the option (1 or 2): "))
        if choice == 1 or choice == 2:
            break
        else:
            print("Invalid Input. Try again.")
    return choice

def txt_or_img():
    while True:
        print("Choose the type of watermark:")
        print("1. Text")
        print("2. Image")
        choice = int(input("Enter the option (1 or 2): "))
        if choice == 1 or choice == 2:
            break
        else:
            print("Invalid Input. Try again.")
    return choice

In [ ]:
cap = cv2.VideoCapture('process.mp4')

In [ ]:
Frame_count=int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
print('No. of Frames = ', Frame_count)

height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
print('Height', height, 'Width', width)

fps = cap.get(cv2.CAP_PROP_FPS)
print('FPS : ', round(fps, 2))

In [ ]:
text_or_image= txt_or_img()
k=True
if text_or_image == 1:
    choice1 = txtwatermark_type()
    text = input("Enter the text for the watermark: ")
    if choice1 == 1:
        hollow_watermark(text)
    elif choice1 == 2:
        filled_watermark(text)
    
    choice2 = txtwatermark_placement()
    if choice2 == 1:
        img_path = 'watermark.png'
        logo = cv2.imread(img_path, -1)
        watermark = image_resize(logo, height=100)
        watermark = cv2.cvtColor(watermark, cv2.COLOR_BGR2BGRA)
    elif choice2 == 2:
        img_path = 'watermark.png'
        logo = cv2.imread(img_path, -1)
        watermark_width=width
        watermark = image_resize(logo, width= int(watermark_width))
        watermark = cv2.cvtColor(watermark, cv2.COLOR_BGR2BGRA)
        k=False
        
elif text_or_image == 2:
    while True:
        image_url = input("Enter the url of image: ")
        response = requests.get(image_url)
        if response.status_code == 200:
            image_data = response.content
            image_path = "watermark.png"
            with open(image_path, "wb") as image_file:
                image_file.write(image_data)
            print(f"Image downloaded and saved as {image_path}")
            break
        
        else:
            print(f"Failed to download image. Status code: {response.status_code}")
    
    img_path = 'watermark.png'
    logo = cv2.imread(img_path, -1)
    watermark = image_resize(logo, height=100)
    watermark = cv2.cvtColor(watermark, cv2.COLOR_BGR2BGRA)
        
opacity = int(input("Enter the opacity percentage: "))/100

In [ ]:
frames=[]
for frame in range(Frame_count):
    ret, img = cap.read()
    if ret == False:
        break
    img = cv2.cvtColor(img, cv2.COLOR_BGR2BGRA)
    overlay = np.zeros((int(height), int(width), 4), dtype='uint8')
            
    watermark_h, watermark_w, watermark_c = watermark.shape
    for i in range(0, watermark_h):
        for j in range(0, watermark_w):
            if watermark[i,j][3] != 0:
                offset = 10
                h_offset = int((height - watermark_h - offset) if k else (height - watermark_h)/2)
                w_offset = int((width - watermark_w - offset) if k else (width - watermark_w)/2)
                overlay[h_offset + i, w_offset+ j] = watermark[i,j]
            
    cv2.addWeighted(overlay, opacity, img, 1.0, 0, img)
            
    img = cv2.cvtColor(img, cv2.COLOR_BGRA2BGR)    
    frames.append(img)
    f=Frame_count - len(frames)
    print("Frames Left: ", f)

In [ ]:
# #If you want to see every frame

# for img in frames:
#     plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))  # Convert BGR to RGB for Matplotlib
#     plt.axis('off')  # Turn off axis labels
#     plt.show()

In [ ]:
output_file = input("Enter the name of output file: ")

filename, ext = os.path.splitext(output_file)
if ext in VIDEO_TYPE:
    fourcc = VIDEO_TYPE[ext]
else:
    print("Sorry. Invalid Extension. Video output will be in .mp4")
    output_file=filename+ ".mp4"
    fourcc = VIDEO_TYPE['.mp4']
    
out = cv2.VideoWriter("process2.mp4", fourcc, fps,(int(width), int(height)))
for img in frames:
    out.write(img)

out.release()
subprocess.run(['ffmpeg', '-i', "process2.mp4" , '-i', 'audio.mp3', '-c:v', 'copy', '-c:a', 'aac', '-strict', 'experimental', output_file])
cap.release()
cv2.destroyAllWindows()


In [ ]:
ipd.Video(output_file)

In [ ]:
os.remove("process.mp4")
os.remove("process2.mp4")
os.remove("watermark.png")
os.remove("audio.mp3")